In [1]:
import xarray as xr
import xmitgcm
import sandbox as sb

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
tm4 = [-sb.degMinSec2decimal(70,54.0876), 
       sb.degMinSec2decimal(39,50.8542)]
at  = [-sb.degMinSec2decimal(70,53.1670), 
       sb.degMinSec2decimal(39,57.1500)]
line= sb.gcspace(tm4, at, npts=50)

In [3]:
data_dir = '/home/ivana/tutorial_global_oce_latlon/run'
grid_dir = data_dir

In [4]:
ds = xmitgcm.open_mdsdataset(data_dir, 
                             prefix=['ssp', 'T'], 
                             iters=[20])
ds = ds.squeeze()
ds = ds.isel(Z=0)

In [5]:
ssp = ds.ihop_ssp

In [6]:
import numpy as np
import pyinterp
mesh = pyinterp.RTree()

In [7]:
#python nuissances
xc = np.array( ssp.XC.values.tolist() )
yc = np.array( ssp.YC.values.tolist() )
ssp= np.array(ssp.values.tolist())

array([   0.        ,    0.        ,    0.        , ..., 1448.69775391,
       1449.63806152, 1451.13244629])

In [8]:
mesh.packing(
    np.vstack( (xc, yc) ).T,
    ssp.ravel()
)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 90 and the array at index 1 has size 40

In [18]:
np.array( ssp.XC.values.tolist() )

array([  2.,   6.,  10.,  14.,  18.,  22.,  26.,  30.,  34.,  38.,  42.,
        46.,  50.,  54.,  58.,  62.,  66.,  70.,  74.,  78.,  82.,  86.,
        90.,  94.,  98., 102., 106., 110., 114., 118., 122., 126., 130.,
       134., 138., 142., 146., 150., 154., 158., 162., 166., 170., 174.,
       178., 182., 186., 190., 194., 198., 202., 206., 210., 214., 218.,
       222., 226., 230., 234., 238., 242., 246., 250., 254., 258., 262.,
       266., 270., 274., 278., 282., 286., 290., 294., 298., 302., 306.,
       310., 314., 318., 322., 326., 330., 334., 338., 342., 346., 350.,
       354., 358.])

In [9]:
from xmitgcm import llcreader
aste = llcreader.SverdrupASTE270Model()
asteDs = aste.get_dataset(k_chunksize=90)
ads = asteDs.sel(face=2, k=0).isel(time=0)

In [10]:
ads

<xarray.Dataset>
Dimensions:    (i: 270, i_g: 270, j: 270, j_g: 270, k_l: 50, k_p1: 51, k_u: 50)
Coordinates:
    face       int64 2
  * i          (i) int64 0 1 2 3 4 5 6 7 8 ... 262 263 264 265 266 267 268 269
  * i_g        (i_g) int64 0 1 2 3 4 5 6 7 8 ... 262 263 264 265 266 267 268 269
  * j          (j) int64 0 1 2 3 4 5 6 7 8 ... 262 263 264 265 266 267 268 269
  * j_g        (j_g) int64 0 1 2 3 4 5 6 7 8 ... 262 263 264 265 266 267 268 269
    k          int64 0
  * k_u        (k_u) int64 0 1 2 3 4 5 6 7 8 9 ... 40 41 42 43 44 45 46 47 48 49
  * k_l        (k_l) int64 0 1 2 3 4 5 6 7 8 9 ... 40 41 42 43 44 45 46 47 48 49
  * k_p1       (k_p1) int64 0 1 2 3 4 5 6 7 8 9 ... 42 43 44 45 46 47 48 49 50
    niter      int64 ...
    time       datetime64[ns] 2002-02-01
    CS         (j, i) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    SN         (j, i) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    drC        (k_p1) >f8 dask.array<chunksize=(51,), meta=np.ndarray>
    drF        float64 dask.array<chunksize=(), meta=np.ndarray>
    dxC        (j, i_g) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    dxG        (j_g, i) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    dyC        (j_g, i) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    dyG        (j, i_g) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    Depth      (j, i) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    PHrefC     float64 dask.array<chunksize=(), meta=np.ndarray>
    PHrefF     (k_p1) >f8 dask.array<chunksize=(51,), meta=np.ndarray>
    rA         (j, i) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    rAs        (j_g, i) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    rAw        (j, i_g) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    rAz        (j_g, i_g) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    Z          float64 dask.array<chunksize=(), meta=np.ndarray>
    Zp1        (k_p1) >f8 dask.array<chunksize=(51,), meta=np.ndarray>
    rhoRef     float64 dask.array<chunksize=(), meta=np.ndarray>
    XC         (j, i) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    XG         (j_g, i_g) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    YC         (j, i) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    YG         (j_g, i_g) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    hFacC      (j, i) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    hFacS      (j_g, i) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    hFacW      (j, i_g) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    maskC      (j, i) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    maskCtrlC  (j, i) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    maskCtrlS  (j_g, i) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    maskCtrlW  (j, i_g) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    maskInC    (j, i) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    maskInS    (j_g, i) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    maskInW    (j, i_g) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    maskS      (j_g, i) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    maskW      (j, i_g) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    Zl         (k_l) >f8 dask.array<chunksize=(50,), meta=np.ndarray>
    Zu         (k_u) >f8 dask.array<chunksize=(50,), meta=np.ndarray>
Data variables:
    ADVr_SLT   (k_l, j, i) float64 dask.array<chunksize=(50, 270, 270), meta=np.ndarray>
    ADVr_TH    (k_l, j, i) float64 dask.array<chunksize=(50, 270, 270), meta=np.ndarray>
    ADVxHEFF   (j, i_g) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    ADVxSNOW   (j, i_g) float64 dask.array<chunksize=(270, 270), meta=np.ndarray>
    ADVx_SLT   (j, i_g) float64 dask.array<chu

In [11]:
lons = ads.XC
lats = ads.YC
salt = ads.SALT

In [14]:
np.vstack((lons.values.ravel(), lats.values.ravel())).T

array([[  52.        ,   67.51677999],
       [  52.0755248 ,   67.54052503],
       [  52.17479724,   67.56723358],
       ...,
       [-127.82520276,   67.56723358],
       [-127.9244752 ,   67.54052503],
       [-128.        ,   67.51677999]])